In [ ]:
!git clone https://github.com/Acciorocketships/sae.git

Cloning into 'sae'...
remote: Enumerating objects: 662, done.
remote: Counting objects: 100% (662/662), done.
remote: Compressing objects: 100% (279/279), done.
remote: Total 662 (delta 392), reused 641 (delta 376), pack-reused 0 (from 0)
Receiving objects: 100% (662/662), 20.79 MiB | 19.70 MiB/s, done.
Resolving deltas: 100% (392/392), done.


In [ ]:
%cd sae

/content/sae


In [ ]:
!pip install -e


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-e option requires 1 argument


In [ ]:
import torch
from torch import nn, optim
from sae import AutoEncoder

In [ ]:
dim = 10
max_n = 5
hidden_dim = 4
epochs = 50

In [ ]:
model = AutoEncoder(dim=dim, hidden_dim=hidden_dim, max_n=max_n)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
x = torch.randn(5, dim)

In [ ]:
batch = torch.arange(0, 5)

In [ ]:
print(batch)

tensor([0, 1, 2, 3, 4])


In [ ]:
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    xr, batchr = model(x, batch)
    loss_data = model.loss()
    loss = loss_data["loss"]

    # Backward pass
    loss.backward()
    optimizer.step()

    # Logging
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

Epoch 10/50, Loss: 46.09587478637695
Epoch 20/50, Loss: 42.90612030029297
Epoch 30/50, Loss: 39.43661117553711
Epoch 40/50, Loss: 35.85036849975586
Epoch 50/50, Loss: 32.07410430908203


In [ ]:
model.eval()
z = model.encoder(x, batch)
xr, batchr = model.decoder(z)

In [ ]:
print(z)

tensor([[ 0.9771,  0.1042, -0.3747, -0.1255],
        [ 0.9713,  0.2305, -0.3769, -0.0842],
        [ 0.9869, -0.0355, -0.3836, -0.0304],
        [ 0.9757, -0.1110, -0.3868, -0.1409],
        [ 0.9729, -0.0326, -0.3840, -0.1135]], grad_fn=<AddBackward0>)


In [ ]:
#To store the resulting embedding for that individual
#Need to change to account for embedding for each individual
individual_embeddings = z.flatten()

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


In [ ]:
num_individuals = 20
num_rows = 5
num_cols = 4

synthetic_embeddings = []
for i in range(num_individuals):
    z = torch.randn(num_rows, num_cols)
    synthetic_embeddings.append(z.flatten())

#Stack embeddings into a single tensor
synthetic_embeddings = torch.stack(synthetic_embeddings)  # Shape: (20, 20)

#Generate random labels (0: healthy, 1: diseased)
labels = torch.randint(0, 2, (num_individuals,))  # Shape: (20,)

In [ ]:
class GeneInteractionNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GeneInteractionNN, self).__init__()
        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = torch.sigmoid(self.output(x))
        return x

In [ ]:
z_train, z_test, y_train, y_test = train_test_split(synthetic_embeddings.detach().numpy(), labels.numpy(), test_size=0.2, random_state=42)

In [ ]:
z_train = torch.tensor(z_train, dtype=torch.float32)
z_test = torch.tensor(z_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [ ]:
input_dim = synthetic_embeddings.shape[1]
hidden_dim = 32
model = GeneInteractionNN(input_dim, hidden_dim)

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(z_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

Epoch 10/100, Loss: 0.6365154385566711
Epoch 20/100, Loss: 0.5977104902267456
Epoch 30/100, Loss: 0.5585681200027466
Epoch 40/100, Loss: 0.5182914733886719
Epoch 50/100, Loss: 0.47572648525238037
Epoch 60/100, Loss: 0.43121540546417236
Epoch 70/100, Loss: 0.3861408531665802
Epoch 80/100, Loss: 0.34172922372817993
Epoch 90/100, Loss: 0.29940494894981384
Epoch 100/100, Loss: 0.2602156698703766


In [ ]:
model.eval()
with torch.no_grad():
    y_pred_test = model(z_test)
    test_loss = criterion(y_pred_test, y_test)
    print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.7851940989494324
